# 2차 과제

## 16조
2019112127 박세호, 2020112167 민장규, 2020112089 최지안

## 데이터 소개
사용한 dataset : https://www.kaggle.com/datasets/kukuroo3/used-car-price-dataset-competition-format

### Feature 소개
- carID : 고유 ID
- brand : 회사 브랜드
- model : 모델명
- year : 출시년도
- transmission : 변속기
- mileage : 주행거리
- fuelType : 연료 유형
- tax : 세금
- mpg : Miles Per Gallon
- engineSize : 엔진 배기량
- price : 가격

## 개요

### 데이터 전처리

- 중복값 처리
- 결측치 처리
- 이상치 처리
- 범주형 변수 이진화

### Linear Regression without sklearn

### Linear Regression with sklearn

### Polynomial Regression

### Modeling nonlinear relationships in the Car Price Dataset

### Model Complexity and Overfitting

### Using regularized methods for regression
- Ridge
- Lasso

# Preprocessing

## 중복값 처리

## 결측치 처리

## 이상치 처리

## 범주형 변수 이진화

# Linear Regression

## Linear Regression without sklearn

## Linear Regression with sklearn

# Polynomial Regression

# Modeling nonlinear relationships in the Car Price Dataset

# Model Complexity and Overfitting

# Using regularized methods for regression

## Ridge

## Lasso